In [ ]:
from quimb import *
from quimb.tensor import *
import quimb as qu
import numpy as np
import quimb.tensor as qtn
import matplotlib.pyplot as plt
import math
import cmath
from numpy.linalg import inv
import cotengra as ctg
import quf

seed_val=10
L_L=256
J=1.
h=1.0

In [ ]:
#qmps, tags=quf.qmps_f(seed_val,L=L_L,in_depth=6,n_Qbit=4, data_type='float64', qmps_type="pollmann", canon="mixed")

qmps, tags=quf.qmera_f(seed_val,L=L_L,in_depth=4,n_Qbit=4,depth_total=int(math.log2(L_L)),data_type='float64',qmera_type='pollmann')


qmps.unitize_(method='mgs')
#qmps.graph(color=tags, figsize=(40, 40), initial_layout='spectral', iterations=500, node_size=300 , edge_scale=10, show_inds=True, show_tags=True)
where=(1,2)
tags = [qmps.site_tag(coo) for coo in where]
print (tags)
mera_ij = qmps.select(tags, which='any')
mera_ij.graph(color=qmps.site_tags, figsize=(40, 40), node_size=300 , iterations=500, edge_scale=10, show_inds=True, show_tags=False)



ERROR! Session/line number was not unique in database. History logging moved to new session 179


In [2]:
ZZ = pauli('Z', dtype="float64") & pauli('Z',dtype="float64")
YY = pauli('Y') & pauli('Y')
XX = pauli('X', dtype="float64") & pauli('X',dtype="float64")
XI = pauli('X',dtype="float64") & pauli('I',dtype="float64")
IX = pauli('I',dtype="float64") & pauli('X',dtype="float64")

#H_ham=0.25*( J*ZZ+h*(XI+XI) )
H_ham = qu.ham_heis(2).real.astype('float64')
H_ham=H_ham.astype('float64')
H_list=[]

for i in range(L_L-1):
 if i==0:
   H_list.append( 0.25*( J*ZZ+h*(2.0*IX+IX) ) )
 elif i==L_L-2:
  H_list.append( 0.25*( J*ZZ+h*(IX+2.0*IX) ) )
 else: 
  H_list.append( 0.25*( J*ZZ+h*(XI+XI) ) )

#H_ham_tn=qtn.Tensor(H_ham.reshape(2, 2, 2, 2), inds=('b0', 'b1','k0', 'k1'), tags=['terms'])

terms = {(i, (i + 1) ): H_list[i] for i in range(L_L-1)}


def local_expectation(mera, terms, where, optimize='auto-hq'):

 tags = [mera.site_tag(coo) for coo in where]
 mera_ij = mera.select(tags, which='any')
 mera_ij_G=mera_ij.gate(terms[where], where)
 mera_ij_ex = (mera_ij_G & mera_ij.H)
 return mera_ij_ex.contract(all, optimize='auto-hq') 

psi=qmps*1.0

def norm_mps(psi):
    # method='qr' is the default but the gradient seems very unstable
    # 'mgs' is a manual modified gram-schmidt orthog routine
    return psi.unitize(method='mgs')


def energy_f_mps(psi, terms, **kwargs):
    """Compute the total energy as a sum of all terms.
    """
    return sum(
        local_expectation(psi, terms, where, **kwargs)
        for where in terms
    )


print (energy_f_mps(psi, terms, optimize='auto-hq'))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-09b3b2a4c224>", line 50, in <module>
    print (energy_f_mps(psi, terms, optimize='auto-hq'))
  File "<ipython-input-3-09b3b2a4c224>", line 44, in energy_f_mps
    return sum(
  File "<ipython-input-3-09b3b2a4c224>", line 45, in <genexpr>
    local_expectation(psi, terms, where, **kwargs)
  File "<ipython-input-3-09b3b2a4c224>", line 31, in local_expectation
    return mera_ij_ex.contract(all, optimize='auto-hq')
  File "/home/reza/quimb/quimb/tensor/tensor_core.py", line 4197, in contract
    return tensor_contract(*self, **opts)
  File "/home/reza/quimb/quimb/tensor/tensor_core.py", line 435, in tensor_contract
    o_array = expression(*(t.data for t in tensors), backend=backend)
  File "/usr/local/lib/python3.8/dist-packages/opt_einsum/contract.py", line 763, in __cal

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-09b3b2a4c224>", line 50, in <module>
    print (energy_f_mps(psi, terms, optimize='auto-hq'))
  File "<ipython-input-3-09b3b2a4c224>", line 44, in energy_f_mps
    return sum(
  File "<ipython-input-3-09b3b2a4c224>", line 45, in <genexpr>
    local_expectation(psi, terms, where, **kwargs)
  File "<ipython-input-3-09b3b2a4c224>", line 31, in local_expectation
    return mera_ij_ex.contract(all, optimize='auto-hq')
  File "/home/reza/quimb/quimb/tensor/tensor_core.py", line 4197, in contract
    return tensor_contract(*self, **opts)
  File "/home/reza/quimb/quimb/tensor/tensor_core.py", line 435, in tensor_contract
    o_array = expression(*(t.data for t in tensors), backend=backend)
  File "/usr/local/lib/python3.8/dist-packages/opt_einsum/contract.py", line 763, in __cal

TypeError: object of type 'NoneType' has no len()

In [ ]:
opt='auto-hq'
tnopt = qtn.TNOptimizer(
     psi,                          # the initial TN
     loss_fn=energy_f_mps,                         # the loss function
     norm_fn=norm_mps,                         # this is the function that 'prepares'/constrains the tn
     constant_tags=[],                 
     tags=[],
     loss_constants={'terms': terms },  # additional tensor/tn kwargs
     loss_kwargs={'optimize': opt},
    autodiff_backend="torch",   # use 'autograd' for non-compiled optimization
    optimizer='L-BFGS-B',     # the optimization algorithm
 )

qmps = tnopt.optimize(n=3000)
